In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
import gym as gym
from time import sleep
from time import time

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/R2_Buf5_1e4/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 1e-4
print(env.action_space)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Discrete(4)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [4]:
def start_method(game_name, save_path=None, restore = False, num_agents = 32,
                 replay_ratio=2, network_type, learning_rate=1e-4, lrdecay=0.99, MAX_STEPS=1000000, DISCOUNT=0.99, NSTEPS=20):
    if save_path == None:
        save_path = "saves/"+game_name+"_"+replay_ratio+"_Replay"
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        #create Global Net and agents.
        coord = tf.train.Coordinator()
        G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_type,lr=learning_rate,decay=lrdecay)
        for agent_num in range(num_agents):
            agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_type))
        saver = tf.train.Saver(max_to_keep=5)

        #restore parameters?
        if restore:
            ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
            saver.restore(sess,ckpt.model_checkpoint_path)
        else:
            sess.run(tf.global_variables_initializer())
        # create a process for every worker
        for thread_id in range(num_agents):
            if thread_id == 0:
                workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", NSTEPS, DISCOUNT, MAX_STEPS, replay_ratio))
                processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR)))
            else:
                workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), NSTEPS, DISCOUNT, MAX_STEPS, replay_ratio))
                processes.append(threading.Thread(target=workers[thread_id].work_acer))
        # run processes as daemon
        for p in processes:
            p.daemon = True
            p.start()

        # join processes after training.
        coord.join(processes)


In [ ]:
tf.reset_default_graph()
start_method(game_name = GAME_NAME, save_path = TB_DIR, restore = false, num_agents=32 replay_ratio=2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_1worker_2  using worker_3  using  2 offline steps, per online step
 2 offline steps, per online step
  using  2 offline steps, per online step
worker_4  using  2 offline steps, per online stepworker_5  using  2 offline steps, per online step

worker_6  using  2 offline steps, per online step
worker_7  using  2 offline steps, per online step
worker_8  using  2 offline steps, per online step
worker_9worker_10  using  2 offline steps, per online step
  using  2 offline steps, per online step
worker_11  using  2 offline steps, per online step
worker_12  using  2 offline steps, per online step
worker_13  using  2 offline steps, per online step
worker_14  using  2 offline steps, per online step
worker_15  using  2 offline steps, per online step
worker_16  using  2 offline steps, per online step
worker_17  using  2 offline steps, per online step
worker_18  using  2 offline steps, per online step
worker_19  using  2 offline steps, per online step
worker_20  using  2 offline steps, per o

/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


worker_eval  using  2 offline steps, per online step
Worker  worker_eval At  859  Running/Max:  1.0 1.0  Frames: 14
pi: [[0.2506794  0.22698486 0.2604526  0.26188314]]
Saving Model
Worker  worker_eval At  30831  Running/Max:  2.1946104608466612 4.0  Frames: 481
pi: [[0.2055129  0.32122043 0.26383388 0.20943284]]
Saving Model
Worker  worker_eval At  60264  Running/Max:  1.974363645723641 4.0  Frames: 646
pi: [[0.18109328 0.34824616 0.31382227 0.15683834]]
Saving Model
Worker  worker_eval At  91003  Running/Max:  2.179865603981069 4.0  Frames: 806
pi: [[0.18714371 0.49554053 0.19783556 0.11948017]]
Saving Model
Worker  worker_eval At  121774  Running/Max:  2.3634860792237644 4.0  Frames: 965
pi: [[0.18136887 0.53312385 0.1447333  0.14077397]]
Saving Model
Worker  worker_eval At  152803  Running/Max:  2.274136909001808 4.0  Frames: 1126
pi: [[0.24081552 0.20726779 0.22808056 0.32383615]]
Saving Model
Worker  worker_eval At  181330  Running/Max:  2.30968982933843 6.0  Frames: 1274
pi: [[0.

Worker  worker_eval At  1659985  Running/Max:  30.46391123334981 44.0  Frames: 8948
pi: [[0.16771966 0.7267294  0.00112348 0.10442752]]
Saving Model
Worker  worker_eval At  1693328  Running/Max:  32.17576809901335 50.0  Frames: 9121
pi: [[0.09578525 0.02206661 0.00592098 0.87622714]]
Saving Model
Worker  worker_eval At  1726037  Running/Max:  32.59237216020081 50.0  Frames: 9291
pi: [[0.13701756 0.5919738  0.02951709 0.24149153]]
Saving Model
Worker  worker_eval At  1754306  Running/Max:  31.90982144976266 50.0  Frames: 9438
pi: [[0.27333686 0.21183899 0.13081393 0.3840103 ]]
Saving Model
Worker  worker_eval At  1770338  Running/Max:  31.918839304786395 50.0  Frames: 9521
pi: [[0.21970114 0.2786529  0.38640735 0.11523858]]
Saving Model
Worker  worker_eval At  1816033  Running/Max:  35.07425983687698 50.0  Frames: 9758
pi: [[1.0579050e-02 2.2895616e-01 2.0438815e-04 7.6026046e-01]]
Saving Model
Worker  worker_eval At  1831050  Running/Max:  34.366833853189284 50.0  Frames: 9836
pi: [[7.